In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd

def load_result_csv_files(directory):
    result_dfs = []
    file_paths = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith("results.csv"):
                file_path = os.path.join(root, filename)
                relative_path = os.path.relpath(file_path, directory)
                df = pd.read_csv(file_path)
                result_dfs.append(df)
                file_paths.append(relative_path)
    return result_dfs, file_paths

directory_path = "/workdir/optimal-summaries-public/_models_ablation"
result_dfs, file_paths = load_result_csv_files(directory_path)
len(result_dfs)

15

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
combined_df = pd.concat(result_dfs, ignore_index=True)
combined_df = combined_df[combined_df["Split"] == "test"].drop(columns=["Split"])
combined_df["Dataset"] = combined_df["Dataset"].replace({"mimic": "MIMIC", "spoken_arabic_digits": "SpokenArabicDigits", "tiselac": "Tiselac"})
combined_df["Model"] = combined_df["Model"].replace({"atomics_sum2atomics_False": "Atomics A", "atomics_sum2atomics_True": "Atomics B", "original": "Original", "shared_encode_time_dim_False": "Shared A", "shared_encode_time_dim_True": "Shared B"})

### Ablation


In [4]:
# Model,Indicators,Summaries,Dataset,Seed,Split,AUC,ACC,F1,Cutoff,Lower threshold,Upper threshold
per_set = combined_df.drop(columns=['Seed', "Cutoff", "Lower threshold", "Upper threshold"]).groupby(["Dataset", "Model", "Indicators", "Summaries"]).agg(["mean", "std"]).round(3)

with open('ablation_per_dataset.tex', 'w') as f:
    tex = per_set.to_latex(escape=True, float_format="{:.2%}".format)
    tex = tex.replace('%', r'\%')
    f.write(tex)

per_set

AUC           ACC         \
                                                    mean    std   mean    std   
Dataset            Model     Indicators Summaries                               
MIMIC              Atomics A False      False      0.910  0.005  0.832  0.008   
                                        True       0.868  0.016  0.852  0.010   
                             True       False      0.910  0.009  0.843  0.002   
                                        True       0.877  0.004  0.856  0.011   
                   Atomics B False      False      0.908  0.002  0.828  0.004   
                                        True       0.909  0.006  0.831  0.010   
                             True       False      0.907  0.014  0.848  0.014   
                                        True       0.910  0.006  0.834  0.013   
                   Original  False      False      0.908  0.002  0.832  0.001   
                                        True       0.875  0.007  0.853  0.011   
                             True       False      0.914  0.003  0.839  0.004   
                                        True       0.869  0.009  0.863  0.002   
                   Shared A  False      False      0.908  0.002  0.835  0.002   
                                        True       0.913  0.003  0.838  0.002   
                             True       False      0.915  0.001  0.842  0.002   
                                        True       0.918  0.004  0.845  0.006   
                   Shared B  False      False      0.905  0.001  0.828  0.001   
                                        True       0.911  0.004  0.838  0.001   
                             True       False      0.916  0.002  0.844  0.002   
                                        True       0.911  0.002  0.837  0.004   
SpokenArabicDigits Atomics A False      False      0.997  0.000  0.941  0.003   
                                        True       0.505  0.006  0.105  0.006   
                             True       False      0.997  0.001  0.946  0.004   
                                        True       0.504  0.003  0.106  0.005   
                   Atomics B False      False      0.997  0.000  0.941  0.003   
                                        True       0.994  0.004  0.921  0.024   
                             True       False      0.997  0.001  0.946  0.004   
                                        True       0.981  0.021  0.848  0.101   
                   Original  False      False      0.992  0.002  0.895  0.005   
                                        True       0.543  0.064  0.133  0.042   
                             True       False      0.992  0.001  0.894  0.005   
                                        True       0.501  0.000  0.101  0.001   
                   Shared A  False      False      0.999  0.000  0.965  0.005   
                                        True       0.999  0.000  0.966  0.001   
                             True       False      0.999  0.000  0.968  0.003   
                                        True       0.999  0.000  0.969  0.002   
                   Shared B  False      False      0.999  0.000  0.965  0.002   
                                        True       0.512  0.004  0.112  0.004   
                             True       False      0.999  0.000  0.973  0.003   
                                        True       0.514  0.006  0.114  0.005   
Tiselac            Atomics A False      False      0.956  0.002  0.720  0.004   
                                        True       0.802  0.262  0.510  0.346   
                             True       False      0.954  0.004  0.711  0.009   
                                        True       0.806  0.265  0.519  0.353   
                   Atomics B False      False      0.956  0.002  0.720  0.004   
                                        True       0.960  0.003  0.732  0.009   
                             True       False      0.954  0.004  0.711  0.009   
   

In [5]:
grouped = combined_df.drop(columns=["Seed", "Dataset", "Cutoff", "Lower threshold", "Upper threshold"]).groupby(["Model", "Indicators", "Summaries"]).agg(["mean", "std"]).round(3)#.reset_index()
# grouped = grouped.style.highlight_max(color = 'green', axis = 0).highlight_min(color = 'red', axis = 0)

with open('ablation_grouped.tex', 'w') as f:
    tex = grouped.to_latex(escape=True, float_format="{:.2%}".format)
    tex = tex.replace('%', r'\%')
    f.write(tex)

grouped

# best auc 0.966000 acc 0.871000 f1 0.857000

AUC           ACC            F1       
                                 mean    std   mean    std   mean    std
Model     Indicators Summaries                                          
Atomics A False      False      0.954  0.038  0.831  0.096  0.820  0.115
                     True       0.725  0.213  0.489  0.367  0.444  0.404
          True       False      0.954  0.038  0.833  0.102  0.820  0.124
                     True       0.729  0.217  0.494  0.370  0.450  0.405
Atomics B False      False      0.954  0.039  0.830  0.096  0.819  0.115
                     True       0.955  0.037  0.828  0.083  0.815  0.106
          True       False      0.953  0.040  0.835  0.103  0.821  0.124
                     True       0.951  0.034  0.808  0.072  0.792  0.094
Original  False      False      0.951  0.036  0.813  0.080  0.803  0.100
                     True       0.691  0.201  0.436  0.373  0.383  0.415
          True       False      0.952  0.034  0.813  0.084  0.802  0.104
                     True       0.727  0.218  0.500  0.379  0.454  0.417
Shared A  False      False      0.956  0.040  0.847  0.097  0.833  0.119
                     True       0.961  0.038  0.862  0.081  0.848  0.100
          True       False      0.958  0.036  0.851  0.098  0.839  0.117
                     True       0.963  0.036  0.865  0.082  0.850  0.103
Shared B  False      False      0.955  0.041  0.844  0.098  0.831  0.119
                     True       0.796  0.214  0.564  0.342  0.527  0.372
          True       False      0.958  0.036  0.852  0.102  0.838  0.123
                     True       0.796  0.213  0.565  0.341  0.527  0.370